In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# 设置数据集目录
data_dir = '/GPUFS/sysu_htlin_6/tester01'

# 定义图像的尺寸和批次大小
img_height, img_width = 224, 224
batch_size = 32

# 使用ImageDataGenerator进行数据增强和准备数据
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# 创建CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# 评估模型性能
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [3]:
# 保存训练好的模型
model.save("Xu.h5")

In [4]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.preprocessing import image

test_set_dir = '/GPUFS/sysu_htlin_6//test set'

# 确保导入了numpy和PIL的Image类
test_image_paths = [os.path.join(test_set_dir, img) for img in os.listdir(test_set_dir)]
test_images = [np.array(Image.open(img_path).resize((224, 224))) / 255.0 for img_path in test_image_paths]
test_images = np.array(test_images, dtype=np.float32)

# 这里假设你的model变量已经是一个训练好的模型
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# 创建类别索引到名称的映射
class_indices = {0: 'Cataract', 1: 'Normal', 2: 'Surgery'}

# 将预测的类别索引转换为类别名称
predicted_labels = [class_indices[cls] for cls in predicted_classes]

results_df = pd.DataFrame({
    'image_name': [os.path.basename(path) for path in test_image_paths],
    'predict': predicted_labels,
    'predict_proba_Normal': predictions[:, 1],
    'predict_proba_Cataract': predictions[:, 0],
    'predict_proba_Surgery': predictions[:, 2]
})

# 输出结果到Excel
results_df.to_excel('Xu.xlsx', index=False)